In [1]:
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

f = open('msnbc.txt')
next(f)
next(f)
names = next(f).rstrip().split()
next(f)
next(f)
next(f)
next(f)
data = []
for row in tqdm(f, total=989818):
    v = np.zeros(len(names), dtype="bool")
    row = row.rstrip().split(" ")
    for each in row:
        each = int(each) - 1
        v[each] = 1
    data.append(v)

data = np.array(data)

data = data[data.sum(axis=1) >= 2]
frame = pd.DataFrame(data, columns = names)
train, test = train_test_split(frame, random_state = 0)
print (len(frame), len(train), len(test))

100%|██████████| 989818/989818 [00:08<00:00, 119374.27it/s]


388434 291325 97109


In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules

frequent_itemsets = fpgrowth(train, use_colnames=True, min_support = 0.01)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(news, living, on-air)",(frontpage),0.013449,0.523312,0.010246,0.761868,1.455857,0.003208,2.001781
1,"(local, news, living)",(frontpage),0.014156,0.523312,0.010868,0.767701,1.467004,0.003460,2.052045
2,"(sports, living)",(frontpage),0.022260,0.523312,0.016511,0.741712,1.417340,0.004862,1.845563
3,"(misc, living)",(frontpage),0.016658,0.523312,0.011980,0.719143,1.374213,0.003262,1.697259
4,"(news, living)",(frontpage),0.044843,0.523312,0.033756,0.752756,1.438444,0.010289,1.928002
5,"(living, news, tech)",(frontpage),0.013198,0.523312,0.010085,0.764109,1.460140,0.003178,2.020798
6,"(business, living)",(frontpage),0.027117,0.523312,0.020829,0.768101,1.467768,0.006638,2.055585
7,"(business, news, living)",(frontpage),0.013494,0.523312,0.011221,0.831595,1.589098,0.004160,2.830602
8,"(misc, summary)",(on-air),0.015189,0.276341,0.012083,0.795480,2.878620,0.007885,3.538333
9,"(business, news, tech)",(frontpage),0.017712,0.523312,0.012677,0.715698,1.367630,0.003408,1.676692


In [3]:
confidences = []
for i, row in rules.iterrows():
    acondition = None
    for name in row.antecedents:
        if acondition is None:
            acondition = (test[name] == 1)
        else:
            acondition = acondition & (test[name] == 1)
    
    ccondition = None
    for name in row.consequents:
        if ccondition is None:
            ccondition = (test[name] == 1)
        else:
            ccondition = ccondition & (test[name] == 1)
    
    confidences.append (len(test[acondition & ccondition]) / len(test[acondition]))

print ("test condidence = %.2f, alen = %.2f, clen = %.2f" % (np.mean(confidences), 
                                                            rules.antecedents.apply(len).mean(), 
                                                            rules.consequents.apply(len).mean()))

test condidence = 0.75, alen = 2.38, clen = 1.00
